In [1]:
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk 

In [2]:
category = 'chest'

In [7]:
from src.data.data_load.data_load_chest import load_data_chest
vol_HU_zyx, spacing_dzyx, meta = load_data_chest(
    '1', modality = "CT"
)
slope = float(meta.get("rescale_slope", 1))
intercept = float(meta.get("rescale_intercept", 0))

In [6]:
from src.data.data_process.data_process import raw_to_attenuation
from src.data.data_process.data_process import reorient_for_axis

In [10]:
vol_zyx = raw_to_attenuation(vol_HU_zyx, slope, intercept)

In [13]:
print("value range (HU)      :", int(vol_HU_zyx.min()), "→", int(vol_HU_zyx.max()))
print("value range (mu)      :", int(vol_zyx.min()), "→", int(vol_zyx.max()))

value range (HU)      : -2048 → 3071
value range (mu)      : 0 → 0


In [14]:
from src.data.data_process.projection import get_projections

In [15]:
[nz, ny, nx] = vol_zyx.shape    
[dz, dy, dx] = spacing_dzyx

In [16]:
geo_dict = {
    # Distances
    "DSD": 1000.0,   # Distance Source Detector (mm)
    "DSO": 600.0,   # Distance Source Origin   (mm)

    # Detector parameters
    "nDetector": np.array([256, 256], dtype=np.int32),   # [nu, nv] = [width, height] (px)
    "dDetector": np.array([2.0, 2.0], dtype=np.float32), # [du, dv] (mm/px)

    # Image parameters
    "nVoxel": np.array([nx, ny, nz], dtype=np.int32),       # [nx, ny, nz]
    "dVoxel": np.array([dx, dy, dz], dtype=np.float32),     # [dx, dy, dz] (mm/vx)

    # Offsets
    "offOrigin":   np.array([0.0, 0.0, 0.0], dtype=np.float32),
    "offDetector": np.array([0.0, 0.0],       dtype=np.float32),

    # Mode & misc
    "mode": "cone",
    "accuracy": 0.5,
    "filter": "ram-lak",   # 供重建算法用，对 Ax 前向投影本身无影响

    # 可选（有的代码不会用到，但保留以便一致）
    "COR": 0.0,                                        # center of rotation (mm)
    "rotDetector": np.array([0.0, 0.0, 0.0], dtype=np.float32),  # (rad)
}



In [17]:
angles_deg = np.arange(0, 360, 3, dtype=np.float32)

In [18]:
from src.data.data_process.projection import get_projections

In [19]:
vol_HU_xyz = np.asarray(vol_HU_zyx, dtype=np.float32).transpose(2, 1, 0)
vol_xyz = np.asarray(vol_zyx, dtype=np.float32).transpose(2, 1, 0)

In [20]:
projs_HU = get_projections(vol_HU_xyz, geo_dict, angles_deg)
projs = get_projections(vol_xyz, geo_dict, angles_deg)

In [21]:
from vis.proj.proj_vis import save_projs_png_uint8

In [22]:
projs.shape

(120, 256, 256)

In [23]:
projs_HU

array([[[-47.41606 , -46.727524, -46.160515, ..., -42.907085,
         -43.59491 , -44.37981 ],
        [-46.71451 , -47.767303, -47.037537, ..., -43.757183,
         -44.501625, -43.36605 ],
        [-46.163433, -47.034687, -48.13436 , ..., -44.685677,
         -43.529785, -42.31934 ],
        ...,
        [-46.131172, -47.087982, -47.93694 , ..., -44.699215,
         -43.783623, -43.033535],
        [-46.73323 , -47.635555, -47.03467 , ..., -43.81079 ,
         -44.384384, -43.617996],
        [-47.393566, -46.67426 , -46.1607  , ..., -42.846165,
         -43.558666, -44.221992]],

       [[-41.004707, -41.710808, -42.281532, ..., -45.100567,
         -45.826893, -46.3285  ],
        [-40.59958 , -41.2655  , -42.005444, ..., -45.9813  ,
         -46.30931 , -46.525513],
        [-40.377155, -40.614983, -41.431446, ..., -46.179714,
         -46.55862 , -47.2267  ],
        ...,
        [-40.28663 , -40.644566, -41.421597, ..., -46.309757,
         -46.86478 , -47.39594 ],
        [-40

In [26]:
paths = save_projs_png_uint8(
    projs=projs_HU,
    angles_deg=angles_deg,
    category="chest",       #  figs/projections/category/
    save_path="figs/projections/test/" + "projections/",         
    origin="lower",         # 如需倒置改 "lower"
)